In [1]:
# hide pakcage non critical warnings

import warnings

warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [5]:
import matplotlib.pyplot as plt

import scipy.io as sio

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.init as init

from custom_data import DCCPT_data

from config import cfg, get_data_dir, get_output_dir, AverageMeter, remove_files_in_dir
from SDAE import SDAE
from tensorboard_logger import Logger

import os
import random
import numpy as np

import data_params as dp

In [4]:
import devkit.api as dk

In [43]:
net = SDAE(dim=[784, 500, 500, 2000, 10], dropout=0.2, slope=0)

In [44]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [47]:
net

SDAE(
  (enc_0): Linear(in_features=784, out_features=500, bias=True)
  (dec_0): Linear(in_features=500, out_features=784, bias=True)
  (enc_1): Linear(in_features=500, out_features=500, bias=True)
  (dec_1): Linear(in_features=500, out_features=500, bias=True)
  (enc_2): Linear(in_features=500, out_features=2000, bias=True)
  (dec_2): Linear(in_features=2000, out_features=500, bias=True)
  (enc_3): Linear(in_features=2000, out_features=10, bias=True)
  (dec_3): Linear(in_features=10, out_features=2000, bias=True)
  (dropmodule1): Dropout(p=0.2, inplace=False)
  (dropmodule2): Dropout(p=0.2, inplace=False)
  (loss): MSELoss()
)

In [14]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [15]:
lr =0.0001

In [16]:
numlayers = 4
lr = 10
maxepoch = 200
stepsize = 1000

In [17]:
for par in net.base[numlayers-1].parameters():
    par.requires_grad = True

In [18]:
# setting up optimizer - the bias params should have twice the learning rate w.r.t. weights params
bias_params = filter(lambda x: ('bias' in x[0]) and (x[1].requires_grad), net.named_parameters())
bias_params = list(map(lambda x: x[1], bias_params))
nonbias_params = filter(lambda x: ('bias' not in x[0]) and (x[1].requires_grad), net.named_parameters())
nonbias_params = list(map(lambda x: x[1], nonbias_params))

In [19]:
optimizer = optim.SGD([{'params': bias_params, 'lr': 2*lr}, {'params': nonbias_params}],
                      lr=lr, momentum=0.9, weight_decay=0.0, nesterov=True)

scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

In [37]:
from torch.utils.tensorboard import SummaryWriter


In [38]:
writer = SummaryWriter('fashion_mnist_experiment_1')


In [21]:
datadir = get_data_dir("mnist")


In [22]:
trainset = DCCPT_data(root=datadir, train=True, h5=False)

In [23]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)


In [24]:
dataiter = iter(trainloader)

In [25]:
dataiter.next()

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([6, 7, 3, 9, 6, 8, 5, 2, 7, 5, 7, 5, 5, 9, 0, 8, 0, 2, 5, 0, 1, 1, 6, 5,
         7, 3, 7, 4, 8, 9, 5, 3, 3, 5, 7, 2, 2, 0, 8, 1, 3, 3, 7, 0, 1, 7, 7, 6,
         5, 5, 9, 3, 8, 6, 8, 2, 6, 2, 1, 7, 1, 6, 9, 9, 2, 1, 4, 5, 8, 4, 4, 4,
         1, 4, 1, 3, 5, 0, 0, 4, 2, 3, 7, 6, 0, 9, 0, 7, 6, 0, 9, 6, 5, 6, 4, 9,
         0, 9, 7, 3, 2, 8, 5, 5, 7, 8, 3, 2, 6, 4, 4, 1, 4, 0, 0, 7, 1, 3, 1, 5,
         2, 1, 6, 5, 1, 4, 5, 9, 4, 2, 5, 8, 5, 2, 2, 4, 5, 2, 0, 4, 6, 7, 9, 9,
         7, 3, 4, 9, 7, 2, 6, 2, 7, 0, 1, 0, 7, 8, 6, 2, 6, 6, 5, 2, 0, 8, 1, 3,
         0, 3, 7, 7, 8, 8, 0, 2, 6, 9, 7, 1, 8, 6, 9, 8, 8, 5, 3, 1, 5, 4, 5, 8,
         6, 6, 4, 5, 2, 0, 6, 1, 9, 1, 3, 1, 1, 4, 0, 6, 2, 2, 8, 9, 2, 1, 4, 7,
         

In [26]:
images, labels = dataiter.next()


In [27]:
images.shape

torch.Size([256, 784])

In [28]:
torch.tensor(3)

tensor(3)

In [29]:
labels

tensor([3, 0, 0, 8, 3, 1, 4, 8, 1, 2, 2, 6, 2, 3, 5, 3, 1, 0, 1, 5, 7, 5, 0, 1,
        6, 2, 6, 3, 6, 5, 8, 8, 5, 8, 6, 6, 4, 7, 6, 0, 3, 5, 5, 9, 2, 6, 5, 3,
        8, 9, 8, 6, 6, 9, 6, 2, 3, 4, 6, 9, 7, 7, 5, 2, 2, 1, 0, 3, 6, 4, 1, 3,
        3, 8, 2, 2, 3, 2, 8, 6, 8, 4, 4, 5, 3, 6, 1, 2, 7, 3, 8, 9, 1, 3, 9, 6,
        0, 6, 5, 0, 6, 6, 8, 6, 3, 1, 2, 0, 5, 7, 6, 8, 6, 8, 2, 5, 6, 2, 1, 1,
        8, 3, 5, 7, 1, 0, 1, 8, 9, 6, 2, 6, 0, 0, 4, 5, 1, 6, 6, 7, 0, 4, 4, 0,
        1, 2, 6, 8, 5, 5, 9, 3, 2, 4, 9, 5, 8, 5, 4, 4, 4, 7, 4, 5, 5, 4, 5, 3,
        4, 4, 8, 9, 9, 7, 5, 7, 5, 4, 9, 2, 0, 3, 3, 2, 8, 2, 2, 2, 7, 6, 5, 7,
        2, 0, 3, 4, 0, 9, 9, 7, 7, 9, 1, 6, 8, 6, 8, 3, 4, 6, 7, 2, 4, 9, 8, 7,
        9, 7, 8, 7, 0, 0, 8, 2, 1, 0, 2, 4, 8, 6, 5, 3, 2, 6, 1, 7, 3, 2, 9, 5,
        6, 5, 6, 1, 4, 6, 1, 6, 6, 7, 2, 3, 0, 2, 2, 8], dtype=torch.uint8)

In [30]:
def make_dot(var):
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def add_nodes(var):
        if var not in seen:
            if isinstance(var, Variable):
                value = '('+(', ').join(['%d'% v for v in var.size()])+')'
                dot.node(str(id(var)), str(value), fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'previous_functions'):
                for u in var.previous_functions:
                    dot.edge(str(id(u[0])), str(id(var)))
                    add_nodes(u[0])
    add_nodes(var.creator)
    return dot

In [31]:
from graphviz import Digraph

In [32]:
images.shape

torch.Size([256, 784])

In [33]:
from torch.autograd import Variable


In [34]:
inputs = torch.randn(256,1,28,28)
y = net(Variable(inputs), 3)
print(y)

tensor(2.4510e-05, grad_fn=<MeanBackward0>)


In [42]:
images.shape

torch.Size([256, 784])

In [45]:
writer.add_graph(net, (images, torch.tensor(3)))
writer.close()

TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%self : ClassType<SDAE>,
		        %x : Tensor,
		        %index : Tensor):
		    %3 : ClassType<Linear> = prim::GetAttr[name="enc_0"](%self)
		    %weight.1 : Tensor = prim::GetAttr[name="weight"](%3)
		    %bias.1 : Tensor = prim::GetAttr[name="bias"](%3)
		    %6 : ClassType<Linear> = prim::GetAttr[name="enc_1"](%self)
		    %weight.2 : Tensor = prim::GetAttr[name="weight"](%6)
		    %bias.2 : Tensor = prim::GetAttr[name="bias"](%6)
		    %9 : ClassType<Linear> = prim::GetAttr[name="enc_2"](%self)
		    %weight.3 : Tensor = prim::GetAttr[name="weight"](%9)
		    %bias.3 : Tensor = prim::GetAttr[name="bias"](%9)
		    %12 : ClassType<Linear> = prim::GetAttr[name="enc_3"](%self)
		    %weight.4 : Tensor = prim::GetAttr[name="weight"](%12)
		    %bias.4 : Tensor = prim::GetAttr[name="bias"](%12)
		    %15 : ClassType<Linear> = prim::GetAttr[name="dec_3"](%self)
		    %weig